In [ ]:
%matplotlib inline

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm

starry.config.lazy = False
starry.config.quiet = True

Load the Earth:

In [ ]:
inc_true = 70.0
map = starry.Map(ydeg=10, inc=inc_true)
map.load("earth")
map.show()

Generate a rotational light curve:

In [ ]:
y_true = np.array(map[1:, :])
theta = np.linspace(0, 360, 1000)
flux0 = map.flux(theta=theta)
sigma = 0.0025
flux = flux0 + np.random.randn(len(theta)) * sigma
plt.plot(theta, flux);

In [ ]:
map.set_data(flux, C=sigma ** 2)
incs = np.linspace(0, 90, 300)

Assume we know the coefficients (almost) exactly:

In [ ]:
y_err = 1e-2
np.random.seed(1)
map.set_prior(mu=y_true + y_err * np.random.randn(len(y_true)), L=y_err ** 2)

ll = np.zeros_like(incs)
for i, inc in tqdm(enumerate(incs), total=len(incs)):
    map.inc = inc
    ll[i] = map.lnlike(theta=theta)

plt.plot(incs, np.exp(ll - ll.max()))
plt.axvline(inc_true, color="C1");

Assume we know only the total variance:

In [ ]:
map.set_prior(L=np.var(y_true))

ll = np.zeros_like(incs)
for i, inc in tqdm(enumerate(incs), total=len(incs)):
    map.inc = inc
    ll[i] = map.lnlike(theta=theta)

plt.plot(incs, np.exp(ll - ll.max()))
plt.axvline(inc_true, color="C1");